In [1]:
# We now test the daily time series by training it on all of the 2023 data and predicted 2024

In [2]:
import pandas as pd

# Load 2023 
daily_counts_jfk_2023 = pd.read_csv("../data/interim/ts_2023.csv")

# Process into a series:
daily_counts_jfk_2023.index = daily_counts_jfk_2023["pickup_date"]
daily_counts_jfk_2023 = daily_counts_jfk_2023.drop("pickup_date", axis = 1)
daily_counts_jfk_2023.columns = ["num_taxis"]

FileNotFoundError: [Errno 2] No such file or directory: '../data/interim/ts_2023.csv'

In [ ]:
# Load all the 2024 data
import glob

files_2024 = glob.glob("../data/raw/yellow_tripdata_2024*.parquet")

# Load and concatenate all the data into a single data frame:
df_2024 = pd.concat((pd.read_parquet(f) for f in files_2024), ignore_index = True)

In [ ]:
df_2024.head()

In [ ]:
df_2024.shape

In [ ]:
df_2024.isna().sum()

In [ ]:
# Daily time series
df_2024['pickup_date'] = df_2024['tpep_pickup_datetime'].dt.date

# Trips per day
daily_counts_2024 = df_2024.groupby('pickup_date').size()

# Plot
daily_counts_2024.plot(figsize = (12,6))

In [ ]:
type(daily_counts_2024)

In [ ]:
daily_counts_2023

In [ ]:
# Get only the rows in 2024:
df_2024 = df_2024[df_2024['tpep_pickup_datetime'].dt.year == 2024]

In [ ]:
# Daily time series
df_2024['pickup_date'] = df_2024['tpep_pickup_datetime'].dt.date

# Trips per day
daily_counts_2024 = df_2024.groupby('pickup_date').size()

# Plot
daily_counts_2024.plot(figsize = (12,6))

In [ ]:
# Get JFK Airport data
df_jfk_2024 = df_2024[df_2024["PULocationID"] == 132].copy() 

In [ ]:
df_jfk_2024.head()

In [ ]:
df_jfk_2024.shape

In [ ]:
# Save this as a .parquet to avoid long reload times in future
df_jfk_2024.to_parquet("../data/interim/processed_jfk_2024.parquet")

# To reload:
# df_jfk_2024 = pd.read_parquet("../data/interim/processed_jfk_2024.parquet")

In [ ]:
# Daily time series
df_jfk_2024['pickup_date'] = df_jfk_2024['tpep_pickup_datetime'].dt.date

# Trips per day
daily_counts_jfk_2024 = df_jfk_2024.groupby('pickup_date').size()

# Plot
daily_counts_jfk_2024.plot(figsize = (12,6))

In [ ]:
df_jfk_2024['pickup_hour'] = df_jfk_2024['tpep_pickup_datetime'].dt.hour

In [ ]:
# sum by day by hour
by_day_hour_jfk_2024 = df_jfk_2024.groupby(['pickup_date', 'pickup_hour']).size().reset_index()

In [ ]:
type(by_day_hour_jfk_2024)

In [ ]:
# Save the hourly counts as a csv
by_day_hour_jfk_2024.to_csv("../data/interim/ts_hour_2024.csv")


In [ ]:
type(daily_counts_jfk_2024)

In [ ]:
print(pd.Series is pd.core.series.Series)  # True

In [ ]:
# Save the daily counts as a csv
daily_counts_jfk_2024.to_csv("../data/interim/ts_2024.csv")

# Reload
# ts_2024 = pd.read_csv("../data/interim/ts_2024.csv")

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

y = daily_counts_jfk_2023

# When forecasting we need the index to have a frequency, for us this is daily
y.index = pd.date_range(start=y.index[0], periods=len(y), freq="D")


dp = DeterministicProcess(
    index = y.index,
    constant = True,   # Dummy feature for bias (y-intercept)
    order = 3,         # Polynomial trend (degree 1 = linear)
    seasonal = True,    # Adds seasonal dummies
    period = 7,        # Weekly seasonality (7-day cycle)
    drop = True,       # Drop first column to avoid collinearity
)

X = dp.in_sample()


In [ ]:
# We now add in the lag features.
# The reason we haven't used all the significant lags is we will need to drop the rows that
# contain null values and if we use lag say 49 we will be dropping about 15% of our data

for i in [1, 2, 3, 4, 6, 7, 8, 13, 14, 15]:
    X[f'y_lag_{i}'] = y.shift(i)

# Drop all na rows
mask = X.notna().all(axis=1) # keep only rows with no NaNs
X = X.loc[mask]
y = y.loc[mask]

In [ ]:
from sklearn.linear_model import LinearRegression
# Fit the model:

model = LinearRegression(fit_intercept = False)
_ = model.fit(X,y)

In [ ]:
# The problem with the above is it is technically cheating as we are assuming we know all the lags in advaance, a more realistic exercise is to try and forecast from our 2023 data itself.
# Meaning we will have to use our own model predicitons to generate the lags:

# Lags we used to train the model
lags = [1, 2, 3, 4, 6, 7, 8, 13, 14, 15]
def forecast(model, y, lags, steps):
    """
    model  = trained linear regression
    y      = pandas Series with historical values
    lags   = list of lags used in training (e.g. [1,2,3])
    steps  = how many steps ahead to forecast
    """
    
    preds = []
    y_hist = y.copy()

    # Create the deterministic features for the forecast
    X_future_det = dp.out_of_sample(steps = steps)

    for i in range(steps): 

        # Get the deterministic row
        x_next = X_future_det.iloc[i].copy()
        
        # Create the lags using historical data
        for j in lags:
            x_next[f'y_lag_{j}'] = y_hist.iloc[-j]
        
        # Predict - x_next is a pandas series and needs to be converted to a dataframe for predictions
        y_pred = model.predict(pd.DataFrame([x_next], columns = x_next.index))[0]
        
        # Append prediction to history so it can be used for future lags
        new_point = pd.Series(y_pred, index=[X_future_det.index[i]])
        y_hist = pd.concat([y_hist, new_point])

        # Add prediction to preds series
        preds.append(new_point)

    # Turn preds into a pandas series
    preds = pd.concat(preds)
    return preds


In [ ]:
# For forecasting we need y as a Series not a DataFrame:
y = y.iloc[:, 0]


In [ ]:
print(forecast(model, y, lags, 10))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error


# Create forecasts, plot and compare to naive baseline
def test_forecasts(steps, y_test, y_hist, model):
    """
    steps = array of the step lengths to forecast
    y_test = pd.Series of the true future values
    y_hist = pd.Series of historical values
    model = model to use for forecasting
    """

    # Compute naive predictions
    # Today = yesterday
    y_pred_naive = y_test.shift(1)
    y_pred_naive.iloc[0] = y_hist.iloc[-1]
    
    print(y_pred_naive) 
    
    for step in steps:
        # Forecast
        y_fore = forecast(model, y_hist, lags, step)

        # Get real values
        y_real = y_test.iloc[0:step]

        # Compute MAE
        mae = mean_absolute_error(y_fore, y_real)
        print(f"MAE: {mae:.2f} for step = {step}")
        
        # Compute naive MAE
        y_step_pred_naive = y_pred_naive.loc[y_real.index]
         
        mae_naive = mean_absolute_error(y_real, y_step_pred_naive)
        print(f"Naive MAE: MAE = {mae_naive:.2f}\n")

        # Plot
        ax = y_real.plot(color='0.25', style='.', title=f"Forecast steps: {step}")
        ax = y_fore.plot(ax=ax, label="Forecast")
        ax = y_step_pred_naive.plot(ax = ax, label = "Naive")
        ax.legend()
        plt.xticks(rotation = 90)
        plt.show()
       

In [ ]:
# Run these test forecasts:
steps = [1, 2, 3, 7, 14, 28, 30, 60, 180, 365]

y_test = daily_counts_jfk_2024

test_forecasts(steps, y_test, y, model)

Based on the above I now have the suspicision that the deterministic part of the linear regression is causing problems with the forecast. We will now make new model using only lags and redo the forecasting to compare. 

In [ ]:
# Create lag design matrix and target series
y_lag = daily_counts_jfk_2023.copy()

# When forecasting we need the index to have a frequency, for us this is daily
y_lag.index = pd.date_range(start=y_lag.index[0], periods=len(y_lag), freq="D")

# Create emty lag data frame
X_lag = pd.DataFrame(index = y_lag.index)

lags = [1, 2, 3, 4, 6, 7, 8, 13, 14, 15]
for i in lags:
    X_lag[f'y_lag_{i}'] = y.shift(i)

# Drop all na rows
mask = X_lag.notna().all(axis=1) # keep only rows with no NaNs
X_lag = X_lag.loc[mask]
y_lag = y_lag.loc[mask]


In [ ]:
from xgboost import XGBRegressor

# Fit the model:

# Linear:
model_lag = LinearRegression()
model_lag.fit(X_lag, y_lag);

# XGBoost:
model_xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8
)
model_xgb.fit(X_lag, y_lag);

In [ ]:
# The problem with the above is it is technically cheating as we are assuming we know all the lags in advaance, a more realistic exercise is to try and forecast from our 2023 data itself.
# Meaning we will have to use our own model predicitons to generate the lags:

def forecast_lag(model, y, lags, steps):
    """
    model  = trained linear regression
    y      = pandas Series with historical values
    lags   = list of lags used in training (e.g. [1,2,3])
    steps  = how many steps ahead to forecast
    """
    
    preds = []
    y_hist = y.copy()

    for i in range(1, steps + 1): 

        # Create the next row
        next_index = y_hist.index.freq  + y_hist.index[-1]
       
        next_index = next_index.date() # convert to date
        x_next = pd.DataFrame(index = [next_index])
        
        # Create the lags using historical data
        for j in lags:
            x_next[f'y_lag_{j}'] = y_hist.iloc[-j]

        
        # Predict - x_next is a pandas series and needs to be converted to a dataframe for predictions
        y_pred = model.predict(x_next)[0]
        
        # Append prediction to history so it can be used for future lags
        new_point = pd.Series(y_pred, index=x_next.index)
        y_hist = pd.concat([y_hist, new_point])

        # Reset y_hist's index
        y_hist.index = pd.date_range(start=y_hist.index[0], periods=len(y_hist), freq="D")


        # Add prediction to preds series
        preds.append(new_point)

    # Turn preds into a pandas series
    preds = pd.concat(preds)
    return preds


In [ ]:
# Create forecasts, plot and compare to naive baseline
def test_forecasts_lag(steps, y_test, y_hist, model):
    """
    steps = array of the step lengths to forecast
    y_test = pd.Series of the true future values
    y_hist = pd.Series of historical values
    model = model to use for forecasting
    """

    # Compute naive predictions
    # Today = yesterday
    y_pred_naive = y_test.shift(1)
    y_pred_naive.iloc[0] = y_hist.iloc[-1]
    
    
    for step in steps:
        # Forecast
        y_fore = forecast_lag(model, y_hist, lags, step)
        y_fore.index.name = "pickup_date"
       
        
        # Get real values
        y_real = y_test.iloc[0:step]

        print(y_fore)

        # Compute MAE
        mae = mean_absolute_error(y_fore, y_real)
        print(f"MAE: {mae:.2f} for step = {step}")
        
        # Compute naive MAE
        y_step_pred_naive = y_pred_naive.loc[y_real.index] 
        mae_naive = mean_absolute_error(y_real, y_step_pred_naive)
        print(f"Naive MAE: MAE = {mae_naive:.2f}\n")


        # Plot
        ax = y_real.plot(color='0.25', style='.', title=f"Forecast steps: {step}")
        ax = y_fore.plot(ax=ax, label="Forecast")
        ax = y_step_pred_naive.plot(ax = ax, label = "Naive")
        ax.legend()
        plt.xticks(rotation = 90)
        plt.show()
       

In [ ]:
# Run these test forecasts:
steps = [1, 2, 3, 7, 14, 28, 30, 60, 180, 365]

y_test = daily_counts_jfk_2024
y_hist = daily_counts_jfk_2023.iloc[:, 0]
y_hist.index = pd.date_range(start=y_hist.index[0], periods=len(y_hist), freq="D")

test_forecasts_lag(steps, y_test, y_hist, model_xgb)

It would now be interesting to do these forecast plots for both XGBoost and the linear model with some variation in the parameters just so we can see who is doing what in terms of performance.

In [ ]:
# Preprocess the data for the models:

def preprocess_linear(lags, order):
    y = daily_counts_jfk_2023

    # When forecasting we need the index to have a frequency, for us this is daily
    y.index = pd.date_range(start=y.index[0], periods=len(y), freq="D")


    dp = DeterministicProcess(
        index = y.index,
        constant = True,   # Dummy feature for bias (y-intercept)
        order = order,         # Polynomial trend (degree 1 = linear)
        seasonal = True,    # Adds seasonal dummies
        period = 7,        # Weekly seasonality (7-day cycle)
        drop = True,       # Drop first column to avoid collinearity
    )

    X = dp.in_sample()

    # We now add in the lag features.
    # The reason we haven't used all the significant lags is we will need to drop the rows that
    # contain null values and if we use lag say 49 we will be dropping about 15% of our data
    
    for i in lags:
        X[f'y_lag_{i}'] = y.shift(i)
    
    # Drop all na rows
    mask = X.notna().all(axis=1) # keep only rows with no NaNs
    X = X.loc[mask]
    y = y.loc[mask]

    return (X, y, dp)
        

def preprocess_non_linear(lags):
    # Create lag design matrix and target series
    y_lag = daily_counts_jfk_2023.copy()
    
    # When forecasting we need the index to have a frequency, for us this is daily
    y_lag.index = pd.date_range(start=y_lag.index[0], periods=len(y_lag), freq="D")
    
    # Create emty lag data frame
    X_lag = pd.DataFrame(index = y_lag.index)
    
    for i in lags:
        X_lag[f'y_lag_{i}'] = y.shift(i)
    
    # Drop all na rows
    mask = X_lag.notna().all(axis=1) # keep only rows with no NaNs
    X_lag = X_lag.loc[mask]
    y_lag = y_lag.loc[mask]

    # Create day of the week feature:
    #X_lag["day_of_week"] = X_lag.index.dayofweek
    #X_lag["is_weekend"] = (X_lag.index.dayofweek >= 5).astype(int)


    return (X_lag, y_lag)

    

In [ ]:
# Fit models:

def fit_linear(X,y):
    model = LinearRegression(fit_intercept = False)
    model.fit(X,y)

    return model

def fit_non_linear(X_lab,y_lag):
    # XGBoost:
    model_xgb = XGBRegressor(
        n_estimators=2000,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    )
    model_xgb.fit(X_lag, y_lag);
    return model_xgb

    

In [ ]:
def forecast_linear(model, y, lags, steps, dp):
    """
    model  = trained linear regression
    y      = pandas Series with historical values
    lags   = list of lags used in training (e.g. [1,2,3])
    steps  = how many steps ahead to forecast
    dp     = the deterministic process used
    """
    
    preds = []
    y_hist = y.copy()

    # Create the deterministic features for the forecast
    X_future_det = dp.out_of_sample(steps = steps)

    for i in range(steps): 

        # Get the deterministic row
        x_next = X_future_det.iloc[i].copy()
        
        # Create the lags using historical data
        for j in lags:
            x_next[f'y_lag_{j}'] = y_hist.iloc[-j]
        
        # Predict - x_next is a pandas series and needs to be converted to a dataframe for predictions
        y_pred = model.predict(pd.DataFrame([x_next], columns = x_next.index))[0]
        
        # Append prediction to history so it can be used for future lags
        new_point = pd.Series(y_pred, index=[X_future_det.index[i]])
        y_hist = pd.concat([y_hist, new_point])

        # Add prediction to preds series
        preds.append(new_point)

    # Turn preds into a pandas series
    preds = pd.concat(preds)
    return preds


In [ ]:
def forecast_non_linear(model, y, lags, steps):
    """
    model  = trained linear regression
    y      = pandas Series with historical values
    lags   = list of lags used in training (e.g. [1,2,3])
    steps  = how many steps ahead to forecast
    """
    
    preds = []
    y_hist = y.copy()

    for i in range(1, steps + 1): 

        # Create the next row
        next_index = y_hist.index.freq  + y_hist.index[-1]
       
        next_index = next_index.date() # convert to date
        x_next = pd.DataFrame(index = [next_index])
        
        # Create the lags using historical data
        for j in lags:
            x_next[f'y_lag_{j}'] = y_hist.iloc[-j]

        # Create day of the week feature:
        #x_next["day_of_week"] = next_index.weekday()
        #x_next["is_weekend"] = int(next_index.weekday() >= 5)

        
        # Predict - x_next is a pandas series and needs to be converted to a dataframe for predictions
        y_pred = model.predict(x_next)[0]
        
        # Append prediction to history so it can be used for future lags
        new_point = pd.Series(y_pred, index=x_next.index)
        y_hist = pd.concat([y_hist, new_point])

        # Reset y_hist's index
        y_hist.index = pd.date_range(start=y_hist.index[0], periods=len(y_hist), freq="D")


        # Add prediction to preds series
        preds.append(new_point)

    # Turn preds into a pandas series
    preds = pd.concat(preds)
    return preds


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create forecasts, plot and compare to naive baseline, the key difference is we will now pass a list of linear and non linear models for ease of use
def test_forecasts_dicts(steps, y_test, y_hist, linear_models, non_linear_models, lags):
    """
    steps = array of the step lengths to forecast
    y_test = pd.Series of the true future values
    y_hist = pd.Series of historical values
    linear_models = dict of linear models
    non_linear_models = dict of non linear models
    lags = lags used in the models
    """

    # Compute naive predictions
    # Today = yesterday
    y_pred_naive = y_test.shift(1)
    y_pred_naive.iloc[0] = y_hist.iloc[-1]
    
   
    for step in steps:

        # Store MAE scores for barplot
        mae_scores = {}
        
        # Get real values
        y_real = y_test.iloc[0:step]
        
        # Plot
        ax = y_real.plot(color='0.25', style='.', title=f"Forecast steps: {step}")
        
        # Forecast the linear models:
        for name, value in linear_models.items():
            model = value[0]
            dp = value[1]
            
            # Get forecast
            y_fore_linear = forecast_linear(model, y_hist, lags, step, dp)

            # Compute MAE linear
            mae_linear = mean_absolute_error(y_fore_linear, y_real)
            mae_scores[name] = mae_linear
            print(f"MAE Linear: {mae_linear:.2f} for step = {step}, model = {name}")

            # Add to plot
            ax = y_fore_linear.plot(ax = ax, label = name)
        

        
        # Forecast the non linear models:
        for name, model in non_linear_models.items():
            y_fore_non_linear = forecast_non_linear(model, y_hist, lags, step)
            y_fore_non_linear.index.name = "pickup_date"

            # Compute MAE non linear
            mae_non_linear = mean_absolute_error(y_fore_non_linear, y_real)
            mae_scores[name] = mae_non_linear
            print(f"MAE Non Linear: {mae_non_linear:.2f} for step = {step}, model = {name}")

            # Add to plot
            ax = y_fore_non_linear.plot(ax = ax, label = name)
       

        
       
        # Compute naive MAE
        y_step_pred_naive = y_pred_naive.loc[y_real.index]
         
        mae_naive = mean_absolute_error(y_real, y_step_pred_naive)
        mae_scores["Naive"] = mae_naive
        print(f"Naive MAE: MAE = {mae_naive:.2f}\n")

        # Plot forecasts
        ax = y_step_pred_naive.plot(ax = ax, label = "Naive")
        ax.legend()
        plt.xticks(rotation = 90)
        plt.show()

        # Plot MAE bar plots:
        df_mae = pd.DataFrame(list(mae_scores.items()), columns=["Model", "MAE"]) 

        plt.figure(figsize=(8,5))
        sns.barplot(data=df_mae, x="Model", y="MAE")

        plt.title(f"Model Comparison by MAE, steps = {step}")
        plt.xticks(rotation=45, ha="right")
        plt.show()
       

In [ ]:
# Run forecasts
def run_forecasts(steps, lags, linear_models, non_linear_models):
    y_test = daily_counts_jfk_2024
    y_hist = daily_counts_jfk_2023.iloc[:, 0]
    y_hist.index = pd.date_range(start=y_hist.index[0], periods=len(y_hist), freq="D")
    
    test_forecasts_dicts(steps, y_test, y_hist, linear_models, non_linear_models, lags)


In [ ]:
# Create models and forecasts
steps = [1, 2, 3, 7, 14, 28, 30, 60, 180, 365]
lags =  [1, 2, 3, 4, 6, 7, 8, 13, 14, 15]

#(X, y, dp) = preprocess_linear(lags)
(X_lag, y_lag) = preprocess_non_linear(lags)
#linear_model = fit_linear(X,y)
non_linear_model = fit_non_linear(X_lag, y_lag)

# Create model dicts
linear_models = {}

for i in [0,1,2,3,4,5]:
    (X,y, dp) = preprocess_linear(lags, i)
    linear_models[f"linear_order{i}"] = (fit_linear(X,y), dp)

non_linear_models = {
    "base_non_linear": non_linear_model
}

run_forecasts(steps, lags, linear_models, non_linear_models)